In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
import os

os.chdir(r'D:\Work\Starfm')

### 研究区加载

In [ ]:
shp = geemap.shp_to_ee('roi.shp')
roi = shp.geometry()
buffer = roi.buffer(1000)
bound = buffer.bounds()

In [ ]:
Map.addLayer(bound, {}, 'bound')

### 定义函数

In [ ]:
def get_weights(i):
    w = ee.Number(windows)
    zero = ee.List.repeat(0, w.multiply(w));
    re_list = zero.set(i, 1)
    kernel = ee.Array(re_list).reshape([w, w]).toList()
    return kernel

def img_convolve(i, collection):
    collection = ee.List(collection)
    w = ee.Number(windows)
    x = ee.Number(-(windows//2))
    y = ee.Number(-(windows//2))
    weights = get_weights(i)
    Kernel = ee.Kernel.fixed(w, w, weights, x, y)
    img = ee.Image(collection.get(0))
    img1 = img.convolve(Kernel)
    return collection.add(img1)

#### Starfm函数

In [ ]:
def Starfm(image_hacc, image_lacc, image_lpre):
    c1 = image_lpre.subtract(image_lacc)
    c2 = image_hacc.subtract(modis_img1)
    seq = ee.List.sequence(0, windows*windows-1, 1)
    f1 = ee.List(seq.iterate(img_convolve, ee.List([c1]))).slice(1)
    f2 = ee.List(seq.iterate(img_convolve, ee.List([c2]))).slice(1)
    f3 = ee.List(seq.iterate(img_convolve, ee.List([image_hacc]))).slice(1)
    def img_weight(i):
        w = windows
        wc = windows//2
        i = ee.Number(i)
        x = i.divide(w).int()
        y = i.mod(w)
        d = ee.Number(1).add(x.subtract(wc).pow(2).add(y.subtract(wc).pow(2)).pow(0.5).divide(wc))
        img1 = ee.Image(f1.get(i)).abs().add(2).log()
        img2 = ee.Image(f2.get(i)).abs().add(2).log()
        mask1 = ee.Image(f1.get(i)).subtract(c1).abs().lte(c1.abs().divide(10))
        mask2 = ee.Image(f2.get(i)).subtract(c2).abs().lte(c2.abs().divide(10))
        mask3 = ee.Image(f3.get(i)).subtract(image_hacc).abs().lte(image_hacc.abs().divide(10))
        mask = mask1.multiply(mask2).multiply(mask3)
        weight = img1.expression('1 / (img1 * img2 * d) * mask', {
            'img1': img1,
            'img2': img2,
            'd': d,
            'mask': mask
        })
        return weight
    image_weight = ee.List(seq.map(img_weight))
    weight_sum = ee.ImageCollection(image_weight).sum()
    image_normweight = image_weight.map(lambda img: ee.Image(img).divide(weight_sum))
    def img_date(i):
        i = ee.Number(i)
        img1 = ee.Image(f1.get(i))
        img2 = ee.Image(f3.get(i))
        weight = ee.Image(image_normweight.get(i))
        img = img1.expression('(img1 + img2) * weight', {
            'img1': img1,
            'img2': img2,
            'weight': weight
        })
        return img
    image_data = ee.List(seq.map(img_date))
    image_hpre = ee.ImageCollection(image_data).sum()
    return image_hpre

### Starfm使用

In [ ]:
windows = 49          # 窗口大小
scale = 10            # 像元大小

##### 定义基准影像

In [ ]:
# 高分辨率影像
s2 = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterBounds(bound)\
            .filterDate('2022-11-07', '2022-11-08')
s2_img = s2.mosaic().select(['B4', 'B8']).reproject(crs='EPSG:4326', scale=scale).clip(bound)    
# 低分辨率影像
modis1 = ee.ImageCollection('MODIS/061/MOD09GQ').filterDate('2022-11-07', '2022-11-08')
modis_img1 = modis1.first().select(['sur_refl_b01', 'sur_refl_b02'], ['B4', 'B8']).reproject(crs='EPSG:4326', scale=scale).clip(bound)
# 需要选择相同的波段，并重新投影到相同的投影和像元大小，以及裁剪到相同的区域

##### 单张影像使用

In [ ]:
modis2 = ee.ImageCollection('MODIS/061/MOD09GQ').filterDate('2022-08-09', '2022-08-10')
modis_img2 = modis2.first().select(['sur_refl_b01', 'sur_refl_b02'], ['B4', 'B8']).reproject(crs='EPSG:4326', scale=scale).clip(bound)
img_modis_to_s2 = Starfm(s2_img, modis_img1, modis_img2)

##### 对影像集使用

In [ ]:
mod_col = ee.ImageCollection('MODIS/061/MOD09GQ').filterDate('2022-08-09', '2022-08-14')
mod_col_re = mod_col.map(lambda img: img.select(['sur_refl_b01', 'sur_refl_b02'], ['B4', 'B8'])
                         .reproject(crs='EPSG:4326', scale=scale).clip(bound))
mod_col_to_s2 = mod_col_re.map(lambda img: Starfm(s2_img, modis_img1, img))

In [ ]:
Map.addLayer(mod_col_to_s2.first(), {}, 'mod2S2')